参考资料：

https://www.kaggle.com/startupsci/titanic-data-science-solutions

主要介绍了 Seaborn 绘图。

1、FacetGrid 的用法；

+ plt.hist

+ sns.pointplot

+ sns.barplot 默认计算平均值

2、


In [194]:
import pandas as pd
import numpy as np
import random as rnd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [273]:
train_df = pd.read_csv('./input/train.csv')
test_df = pd.read_csv('./input/test.csv')

In [274]:
combine = pd.concat([train_df, test_df])
combine.shape

(1309, 12)

In [275]:
train_df.shape

(891, 12)

In [276]:
test_df.shape

(418, 11)

In [277]:
train_df.drop(['Ticket', 'Cabin'], axis=1, inplace=True)
test_df.drop(['Ticket', 'Cabin'], axis=1, inplace=True)

In [278]:
combine = pd.concat([train_df, test_df])

In [279]:
train_df.shape

(891, 10)

In [280]:
test_df.shape

(418, 9)

In [281]:
# The expand=False flag returns a DataFrame.
combine['Title'] = combine.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
# pd.crosstab(combine['Title'], combine['Sex'])

把 Title 再合并一下。

In [282]:
combine['Title'] = combine['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

combine['Title'] = combine['Title'].replace('Mlle', 'Miss')
combine['Title'] = combine['Title'].replace('Ms', 'Miss')
combine['Title'] = combine['Title'].replace('Mme', 'Mrs')

# combine[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [283]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

combine['Title'] = combine['Title'].map(title_mapping)
# combine['Title'].value_counts(dropna=None)

In [284]:
combine.head()

,Age,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Title
0,22.0,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,1
1,38.0,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,3
2,26.0,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,2
3,35.0,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,3
4,35.0,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,1


In [285]:
combine.drop(['Name', 'PassengerId', 'Survived'], axis=1,inplace=True)

In [286]:
combine['Sex'] = combine['Sex'].map( {'female': 0, 'male': 1} )

In [287]:
combine['Age'].fillna(combine['Age'].median(), inplace=True)

分组估计年龄

用 transform 函数试试看。

```python
guess_ages = np.zeros((2,3))
guess_ages

for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

train_df.head()
```

In [288]:
combine['Age'].min(), combine['Age'].max()

(0.17, 80.0)

In [289]:
out, bins = pd.cut(combine['Age'], 5, retbins=True)
bins

array([ 0.09017, 16.136  , 32.102  , 48.068  , 64.034  , 80.     ])

In [290]:
def age_cut(age):
    if age <= 16:
        return 0
    if age <= 32:
        return 1
    if age <= 48:
        return 2
    if age <= 64:
        return 3
    return 4

In [291]:
combine['Age'] = combine['Age'].map(age_cut)    

In [292]:
combine['FamilySize'] = combine['SibSp'] + combine['Parch'] + 1

In [293]:
combine['IsAlone'] = (combine['FamilySize'] == 1).astype(int)

In [294]:
# combine[(combine['Parch'] == 0) & (combine['SibSp'] == 0)]

In [295]:
combine.drop(['Parch', 'SibSp', 'FamilySize'], axis=1, inplace=True)

In [296]:
combine['Age*Class'] = combine.Age * combine.Pclass

In [297]:
freq_port = combine.Embarked.dropna().mode()[0]
freq_port
combine['Embarked'].fillna(freq_port,inplace=True)

In [298]:
combine['Embarked'] = combine['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} )

In [299]:
combine['Fare'].fillna(combine['Fare'].dropna().median(), inplace=True)

In [303]:
out, bins = pd.qcut(combine['Fare'], 4, retbins=True)
bins

array([  0.    ,   7.8958,  14.4542,  31.275 , 512.3292])

In [304]:
def fare_cut(age):
    if age <=  7.8958:
        return 0
    if age <= 14.4542:
        return 1
    if age <= 31.275:
        return 2
    return 3

In [305]:
combine['Fare'] = combine['Fare'].map(fare_cut)

In [312]:
X = combine.iloc[:train_df.shape[0]]
y = train_df["Survived"]
X_test = combine.iloc[train_df.shape[0]:]

## 下面开始建模

In [393]:
from sklearn.model_selection import cross_val_score

logreg = LogisticRegression()
logreg_scores = cross_val_score(logreg, X, y, cv=10)
logreg_scores_mean = logreg_scores.mean()

In [326]:
logreg.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [348]:
def func_abs(x):
    return abs(x)


def sign(x):
    return int(x >= 0)


pd.Series(
    logreg.coef_[0], index=X.columns).transform([func_abs, sign]).sort_values(
        by='func_abs', ascending=False)

,func_abs,sign
Sex,2.144416,0
Pclass,0.768966,0
Title,0.427808,1
IsAlone,0.405252,1
Embarked,0.296722,1
Age*Class,0.166261,0
Fare,0.125128,1
Age,0.115569,0


In [394]:
svc = SVC()

svc_scores = cross_val_score(svc, X, y, cv=10)
svc_scores_mean = svc_scores.mean()
svc_scores_mean

0.8125922142776074

In [395]:
knn = KNeighborsClassifier(n_neighbors = 3)

knn_scores = cross_val_score(knn, X, y, cv=10)
knn_scores_mean = knn_scores.mean()
knn_scores_mean

0.7959130632164341

In [396]:
gaussian = GaussianNB()

gaussian_scores = cross_val_score(gaussian, X, y, cv=10)
gaussian_scores_mean = gaussian_scores.mean()

In [397]:
perceptron = Perceptron()

perceptron_scores = cross_val_score(perceptron, X, y, cv=10)
perceptron_scores_mean = perceptron_scores.mean()

In [398]:
linear_svc = LinearSVC()

linear_svc_scores = cross_val_score(linear_svc, X, y, cv=10)
linear_svc_scores_mean = linear_svc_scores.mean()

In [399]:
sgd = SGDClassifier()

sgd_scores = cross_val_score(sgd, X, y, cv=10)
sgd_scores_mean = sgd_scores.mean()

In [400]:
decision_tree = DecisionTreeClassifier()

decision_tree_scores = cross_val_score(decision_tree, X, y, cv=10)
decision_tree_scores_mean = decision_tree_scores.mean()

In [401]:
random_forest = RandomForestClassifier(n_estimators=100)

random_forest_scores = cross_val_score(random_forest, X, y, cv=10)
random_forest_scores_mean = random_forest_scores.mean()

In [402]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [svc_scores_mean, knn_scores_mean, logreg_scores_mean, 
              random_forest_scores_mean, gaussian_scores_mean, perceptron_scores_mean, 
              sgd_scores_mean, linear_svc_scores_mean, decision_tree_scores_mean]})
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,0.812680
0,Support Vector Machines,0.812592
8,Decision Tree,0.810445
2,Logistic Regression,0.796823
1,KNN,0.795913
7,Linear SVC,0.790069
4,Naive Bayes,0.766648
6,Stochastic Gradient Decent,0.727108
5,Perceptron,0.663250


In [403]:
random_forest.fit(X,y)
y_pred =random_forest.predict(X_test)

In [408]:
from xgboost import XGBClassifier

xgbc = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5)
xgbc.fit(X, y)
y_pred = xgbc.predict(X_test)
# 0.80861

In [409]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": y_pred
    })
submission.to_csv('./submission_xgbc.csv', index=False)